In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [2]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torchvision.datasets import ImageFolder
import os
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pretrainedmodels
from adamW import AdamW
#from lr_finder import LRFinder

In [3]:
#Example code for only one of my attribute training sets
path = 'Painter/PaintingsDataset'

In [7]:
from PIL import Image
#NOTE: CHANGE THIS IF YOU WANT TO SWITCH DIR
#removes images from training set that are corrupt.
images_path = 'Painter/PaintingsDataset'
image_extensions = ['.jpg', '.png', '.jpeg']
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path)
         for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]

for i, item in enumerate(images):
    try:
        im = Image.open(item)
        if item.split('.')[-1] == 'png':
            rgb_im = im.convert('RGB')
            os.remove(item)
            print('removed png' + item)
            rgb_im.save(item.split('.')[0] + '.jpg')
    except IOError:
        os.remove(item)
        print('removed '+ item)

In [5]:
#this is necessary because all of the images in my training set are symbolic links
def loadsymlink(path):
    returnImage = Image.open(os.readlink(path))
    return returnImage.convert('RGB')

In [4]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
        transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
        ]),
    'valid':
        transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize
        ])
}

image_datasets = {
    'train':
        ImageFolder(os.path.join(path,'train'), data_transforms['train']),
    'valid':
        ImageFolder(os.path.join(path,'valid'), data_transforms['valid'])}

dataloaders = {
    'train':
        torch.utils.data.DataLoader(
            image_datasets['train'],
            batch_size=8,
            shuffle=True,
            num_workers=4),
    'valid':
        torch.utils.data.DataLoader(
            image_datasets['valid'],
            batch_size=8,
            shuffle=False,
            num_workers=4)}

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet34(pretrained=True).to(device)

#good LR always happens to be small for AdamW
#optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-5)
optimizer = optim.SGD(model.parameters(), lr=5e-3, weight_decay=5e-3)

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 8).to(device)

criterion = nn.CrossEntropyLoss()

print("creating learning rate scheduler")

steps = len(dataloaders['train'])
exp_lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

creating learning rate scheduler


In [8]:
#typically I get my best lr right before the descent in loss, make sure to set lr = 1e-8 before running
lr_finder = LRFinder(model, optimizer, criterion, device='cuda')
lr_finder.range_test(dataloaders['train'], end_lr=1, num_iter=100)
lr_finder.plot()

NameError: name 'LRFinder' is not defined

In [6]:
def train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=20,cycle_mult=1):

    steps = len(dataloaders['train'])
    completed = 0
    
    best_acc = 0.0
    
    dataset_sizes = {'train':len(dataloaders['train'].dataset),'valid':len(dataloaders['valid'].dataset)}
    
    #wandb.watch(model)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
 
        epoch_loss = 1.0

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
 
            running_loss = 0.0
            running_corrects = 0
 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
 
                outputs = model(inputs)
                loss = criterion(outputs, labels)
 
                if phase == 'train':
                    if scheduler is not None:
                        steps -= 1
                        scheduler.step()
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
 
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
 
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
            if phase == 'valid':
                #wandb.log({'epoch_loss':epoch_loss,'epoch_acc':epoch_acc})
                
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = model.state_dict()
        
        
        if scheduler is not None:
            lrval = scheduler.get_lr()[0]
            print('steps: %d lr: %.8f'%(steps,lrval))
            
        if (steps <= 0):
            steps = len(dataloaders['train'])
            print(steps)
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

        print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                    epoch_loss,
                                                    epoch_acc))    
    
    torch.save(best_model_wts, 'vipsave.pt')
    return model

In [7]:
model_trained = train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=20)

Epoch 1/20
----------
valid loss: 1.4778, acc: 0.4559
Epoch 2/20
----------
valid loss: 1.3489, acc: 0.4941
Epoch 3/20
----------
valid loss: 1.3954, acc: 0.4765
Epoch 4/20
----------
valid loss: 1.2959, acc: 0.4882
Epoch 5/20
----------
valid loss: 1.3675, acc: 0.4765
Epoch 6/20
----------
valid loss: 1.3464, acc: 0.5412
Epoch 7/20
----------
valid loss: 1.3070, acc: 0.5176
Epoch 8/20
----------
valid loss: 1.4109, acc: 0.4941
Epoch 9/20
----------
valid loss: 1.5158, acc: 0.4912
Epoch 10/20
----------
valid loss: 1.5125, acc: 0.5206
Epoch 11/20
----------
valid loss: 1.4831, acc: 0.5176
Epoch 12/20
----------
valid loss: 1.5267, acc: 0.5324
Epoch 13/20
----------
valid loss: 1.5896, acc: 0.5059
Epoch 14/20
----------
valid loss: 1.6111, acc: 0.4941
Epoch 15/20
----------
valid loss: 1.6979, acc: 0.4765
Epoch 16/20
----------
valid loss: 1.6867, acc: 0.4676
Epoch 17/20
----------
valid loss: 1.5895, acc: 0.5265
Epoch 18/20
----------
valid loss: 1.6957, acc: 0.4882
Epoch 19/20
-------

In [9]:
model.load_state_dict(torch.load('vipsave.pt'))
torch.save(model_trained, 'PaintingsClassifier.pth')